<a href="https://colab.research.google.com/github/Chetna2912/TTS-/blob/main/English.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install TTS

In [ ]:
!pip install torch

In [ ]:
!pip install git+https://github.com/coqui-ai/TTS.git

  Cloning https://github.com/coqui-ai/TTS.git to /tmp/pip-req-build-m015tdnn
  Running command git clone --filter=blob:none --quiet https://github.com/coqui-ai/TTS.git /tmp/pip-req-build-m015tdnn
  Resolved https://github.com/coqui-ai/TTS.git to commit dbf1a08a0d4e47fdad6172e433eeb34bc6b13b4e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install datasets

**Clone the TTS Repository**

In [ ]:
!git clone https://github.com/coqui-ai/TTS.git

fatal: destination path 'TTS' already exists and is not an empty directory.


In [ ]:
%cd TTS

/content/TTS


In [ ]:
from TTS.api import TTS

tts = TTS(model_name='tts_models/en/ljspeech/glow-tts', progress_bar=True, gpu=False)

 > tts_models/en/ljspeech/glow-tts is already downloaded.
 > vocoder_models/en/ljspeech/multiband-melgan is already downloaded.
 > Using model: glow_tts
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:0
 | > fft_size:1024
 | > power:1.1
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:50.0
 | > mel_fmax:7600.0
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Vocoder Model: multiband_melgan
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resam

In [ ]:
!pip install coqui-tts

**Load Configuration and Prepare for Fine-tuning**

In [ ]:
from datasets import load_dataset

# Load the LJSpeech dataset
dataset = load_dataset("keithito/lj_speech")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import torch
from torch.nn.utils.rnn import pad_sequence

class LJSpeechDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        text = self.dataset[idx]['text']
        audio = self.dataset[idx]['audio']

        # Convert text to tensor (this example uses ASCII values)
        text_tensor = torch.tensor([ord(c) for c in text])  # Adjust based on how you want to encode the text
        return text_tensor, audio

def collate_fn(batch):
    texts, audios = zip(*batch)

    # Pad the text sequences
    texts = pad_sequence(texts, batch_first=True, padding_value=0)  # Use 0 as the padding value
    return texts, audios

# Load the dataset and create DataLoader
ljspeech_dataset = LJSpeechDataset(dataset['train'])
dataloader = DataLoader(ljspeech_dataset, batch_size=config['train']['batch_size'], shuffle=True, collate_fn=collate_fn)


In [ ]:
!pip install gtts

In [ ]:
import os
import torch
import time
from gtts import gTTS , gTTSError
from TTS.config import load_config
from TTS.utils.audio import AudioProcessor
from torch.utils.data import DataLoader, Dataset
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

#  Load the configuration file
import json

# Load the configuration file
with open("/content/drive/MyDrive/Colab Notebooks/config.json", 'r') as config_file:
    config = json.load(config_file)


# Specify the dataset path
config['dataset']["path"] = "/content/drive/MyDrive/Colab Notebooks/technical_dataset.txt"

# Initialize the audio processor (if needed)
ap = AudioProcessor(**config['audio'])

# Define your Dataset class
class CustomDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        text = self.dataset[idx]['text']
        audio = self.dataset[idx]['audio']
        return text, audio

def text_to_speech(text, output_file):
    tts = gTTS(text=text, lang='en')
    tts.save(output_file)
    print(f"Saved speech to {output_file}")

max_total_attempts = 10  # Total attempts limit
current_attempts = 0  # Counter for attempts

for epoch in range(config['train']['epochs']):
    for idx, (text, _) in enumerate(dataloader):
        text_str = text[0].numpy().tobytes().decode('utf-8')  # Adjust based on your text encoding
        output_file = f"/content/drive/MyDrive/Colab Notebooks/output_{idx}.mp3"

        attempts = 0  # Attempts for the current text
        while attempts < 5 and current_attempts < max_total_attempts:
            try:
                text_to_speech(text_str, output_file)
                break  # Break the loop if successful
            except gTTSError as e:
                print(f"Error: {e}. Retrying...")
                attempts += 1
                current_attempts += 1  # Increment the total attempts

                if attempts < 5:  # Only wait if not exceeding attempts for this text
                    wait_time = 5 * (2 ** attempts)  # Exponential backoff
                    print(f"Waiting for {wait_time} seconds before retrying...")
                    time.sleep(wait_time)  # Wait before retrying

        if attempts == 5:
            print(f"Max retries reached for text at index {idx}. Skipping...")

        if current_attempts >= max_total_attempts:
            print("Maximum total attempts reached. Exiting.")
            break  # Exit the outer loop if total attempts exceeded

    if current_attempts >= max_total_attempts:
        break  # Exit the outer loop if total attempts exceeded

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:8000
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
Error: 429 (Too Many Requests) from TTS API. Probable cause: Unknown. Retrying...
Waiting for 10 seconds before retrying...
E

**Evaluation**

In [ ]:
from TTS.api import TTS
from TTS.utils.audio import AudioProcessor
import numpy as np

# Initialize TTS model
tts = TTS(model_name='tts_models/en/ljspeech/glow-tts', progress_bar=True, gpu=False)

# Define parameters
sample_rate = 22050
frame_length_ms = 50  # e.g., 50 ms
frame_shift_ms = 12.5  # e.g., 12.5 ms
fft_size = 2048  # Increased to be larger than win_length

# Calculate win_length based on sample rate and frame_length_ms
win_length = int(frame_length_ms * sample_rate / 1000)

# Ensure win_length is less than or equal to fft_size
if win_length > fft_size:
    raise ValueError(f"win_length ({win_length}) cannot be larger than fft_size ({fft_size}).")

# Initialize the audio processor with the defined parameters
ap = AudioProcessor(
    sample_rate=sample_rate,
    frame_length_ms=frame_length_ms,
    frame_shift_ms=frame_shift_ms,
    num_mels=80,
    fft_size=fft_size,
    win_length=win_length,
)

test_sentences = [
    "API is used to connect different software systems.",
    "CUDA enables efficient computation on GPUs.",
    "TTS converts text into speech.",
    "OAuth provides secure authorization.",
    "REST defines a set of constraints for web services."
]

# Generate and save audio for each sentence
for sentence in test_sentences:
    audio = tts.tts(sentence)
    audio = np.array(audio)  # Ensure audio is a NumPy array

    # Save audio file
    ap.save_wav(audio, f"{sentence[:10]}.wav")


 > tts_models/en/ljspeech/glow-tts is already downloaded.
 > vocoder_models/en/ljspeech/multiband-melgan is already downloaded.
 > Using model: glow_tts
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:0
 | > fft_size:1024
 | > power:1.1
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:50.0
 | > mel_fmax:7600.0
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


/usr/local/lib/python3.10/dist-packages/TTS/utils/io.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=map_location, **kwargs)


 > Vocoder Model: multiband_melgan
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:0
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:50.0
 | > mel_fmax:7600.0
 | > pitch_fmin:0.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:/root/.local/share/tts/vocoder_models--en--ljspeech--multiband-melgan/scale_stats.npy
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Generator Model: multiband_melgan_generator
 > Discriminator Model: melgan_multiscale_discriminator
 > Setting up Audio Processor...
 | 